In [2]:
import cv2
import numpy as np

def resize_img(img):
    return(cv2.resize(img,(500,500)))
    
def get_output_image(original_image_path,fully_annotated_image_path,partially_annotated_image_path):
    if not isinstance(original_image_path,(str, list)):
        raise TypeError("image path must be either string or list data type")
    if not isinstance(fully_annotated_image_path,(str, list)):
        raise TypeError("image path must be either string or list data type")
    if not isinstance(partially_annotated_image_path,(str, list)):
        raise TypeError("image path must be either string or list data type")
    fully_annotated=cv2.imread(fully_annotated_image_path)#reads the fully annotated image from its path
    fully_annotated=resize_img(fully_annotated)#resizes image
    cv2.imshow('Fully annotated image',fully_annotated)#displays image
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    height,width,channels=fully_annotated.shape
    print(f'Height:{height},weight:{width},channels:{channels}')#prints height, width and color channels
    
    gray_fa=cv2.cvtColor(fully_annotated, cv2.COLOR_BGR2GRAY)#converts image to grayscale
    clahe=cv2.createCLAHE(clipLimit=1.0099,tileGridSize=(8,8))#helps enhance contrast, window size is 8x8
    gray_fa=clahe.apply(gray_fa)

    #cv2.imshow('Enhanced gray scale image of the fully annotated image:', clahe_fa)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    #adaptive_threshold = cv2.adaptiveThreshold(gray_fa, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    gray_fa=cv2.GaussianBlur(gray_fa,(5,5),0)#gaussian blur used to smooth out noise, sd=0,5x5 kernel size
 
    edges=cv2.Canny(gray_fa,50,150)#used for edge detection
    #gradient>150 threshold= strong edges, <50= weak edges
    cv2.imshow('Edge detection',edges)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    kernel=np.ones((3,3),np.uint8)#defines neighbourhood considered for pixel dilation
    edges=cv2.dilate(edges,kernel,iterations=1)#enhances boundaries of image on edge extracted image

    cv2.imshow('Dilation:',edges)
    cv2.waitKey(0)
    cv2.destroyAllWindows()#displays image after dilation

    contours, _=cv2.findContours(edges,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)#finds contours of edges, RETR_LIST collects
    #all contours irrespective of nesting hierarchy
    #print('Number of filtered contours:')
    #print(len(filtered_contours))
    print('Number of total contours:')
    print(len(contours))

    filtered_contours=[]
    for contour in contours:
        if len(contour)>= 5:
            x,y,w,h=cv2.boundingRect(contour)#fits a rectangle to the contour
            aspect_ratio=w/h
            aspect_ratio_threshold=2#calculates threshold value for aspect ratio
            if aspect_ratio<aspect_ratio_threshold:
                filtered_contours.append(contour)#appends to list if condition satisfies
    sorted_filtered_contours=sorted(filtered_contours,key=cv2.contourArea,reverse=True)#amongst the filtered contours, the
    #i=ones having greatest area of put at the start
    dog_contour=sorted_filtered_contours[0]
    fa=fully_annotated.copy()
    print('Dog contour:')
    print(len(dog_contour))#logically, the contour having largest area is chosen as the one to be de annotated

    cv2.drawContours(fa,[dog_contour],-1,(255,0,0),2)
    cv2.imshow('Drawn contours:',fa)#extracted contours drawn over image
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    mask=np.zeros_like(gray_fa) #mask originally kept as a black image
    cv2.drawContours(mask,[dog_contour],-1,(255),thickness=cv2.FILLED)
    #thickness=cv2.FILLED means that annotated region filled with white, zeros initially is a black image with same dimensions as og

    result = cv2.inpaint(fully_annotated,mask,inpaintRadius=5,flags=cv2.INPAINT_TELEA)#filles annotation region
    #telea's method retains underlying structure
    cv2.imshow('Image after partial deannotation',result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    cv2.imwrite(partially_annotated_image_path,result)#stores the image in its path

get_output_image('C:/Personal/ML/datasets/img9/original_image.jpg',
                 'C:/Personal/ML/datasets/img9/fully_annotated_image.jpg',
                 'C:/Personal/ML/datasets/img9/partially_annotated_image.jpg')

get_output_image('C:/Personal/ML/datasets/img13/original_image.jpg',
                 'C:/Personal/ML/datasets/img13/fully_annotated_image.jpg',
                 'C:/Personal/ML/datasets/img13/partially_annotated_image.jpg')

Height:500,weight:500,channels:3
Number of total contours:
98
Dog contour:
705
Height:500,weight:500,channels:3
Number of total contours:
134
Dog contour:
2750
